In [1]:
import config as config
from src.analysis.registery import EEGEpochExtractor
from src.visualizations.p_100_component import P100Plotter

In [3]:
from src.dataset.bids import create_bids_dataset
from src.analysis.registery import EEGEpochExtractor
from src.visualizations.p_100_component import P100Plotter
import config as config

import pdb


if __name__== '__main__':
    
    if config.CREATE_BIDS_DATASET:
        create_bids_dataset(dataset_details=config.filepaths[8:])

    if config.PLOT_P100_COMP:
        epochs_extractor = EEGEpochExtractor()
        
        # Variable values for visual stimulus
        trial_mode, trial_unit, experiment_mode = '', 'Words', 'Experiment' 
        trial_boundary, trial_type, modality= 'Start', 'Stimulus', 'Pictures'
        
        rest_epochs = epochs_extractor.create_epochs(
            trial_mode=trial_mode, trial_type='Fixation',
            trial_unit=trial_unit, experiment_mode=experiment_mode,
            trial_boundary=trial_boundary, modality=modality, tmin=0.2, tmax=0.9
        )
        rest_amplitudes = {}
        for key, epochs in rest_epochs.items():
            subject_id, session_id = key[0], key[1]
            name = f"{trial_mode}_{trial_unit}_{experiment_mode}_{trial_boundary}_{trial_type}"
            name = f'{name}_p_100_component_rest'
            plotter = P100Plotter(channels=['PO3', 'POz', 'PO4'])
            plotter.plot_p100(
                epochs=epochs, name=name, 
                subject_id=subject_id, session_id=session_id)
            rest_amplitudes[key] = plotter.get_p100_amplitude(epochs)
        


        print(rest_amplitudes)

╭────────────────────────────────────╮
│  Initializing VisualEpochExtractor │
╰────────────────────────────────────╯

Loading raw EEG data

Loading sub-01, ses-01

╭─────────────────────────────────────────╮
│ 🚀 Initializing BIDSDatasetReader Class │
╰─────────────────────────────────────────╯

Loading Processed EEG Data: sub-01 ses-01

...

╭────────────────────────────────╮
│  Initializing DataLoader Class │
╰────────────────────────────────╯

Creating EPOCHS

        Extraction Criteria        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Parameter          ┃ Value      ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ 🧪 Trial Mode      ┃            ┃
┃ 📏 Trial Unit      ┃ Words      ┃
┃ 🔗 Experiment Mode ┃ Experiment ┃
┃ 🎭 Trial Boundary  ┃ Start      ┃
┃ 🔬 Trial Type      ┃ Fixation   ┃
┃ 📊 Modality        ┃ Pictures   ┃
┃ ⏱ Tmin             ┃ 0.2        ┃
┃ ⏳ Tmax            ┃ 0.9        ┃
┗━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━┛

Not setting metadata
130 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 130 events and 701 original time points ...
0 bad epochs dropped


ValueError: Baseline interval [-0.2, 0.0] s is outside of epochs data [0.2, 0.9] s. Epochs were probably cropped.